In [49]:
from ultralytics import YOLO
import cv2
import torch

In [50]:
# Carregar o modelo YOLOv8 pré-treinado
model = YOLO("yolov8n.pt")  # Use "yolov8l.pt" para mais precisão

# Captura de vídeo da webcam
cap = cv2.VideoCapture(0)

# ROI (Region of Interest) para detecção de objetos
roi_start = (100, 100)  # Coordenadas do canto superior esquerdo da área
roi_end = (300, 500)    # Coordenadas do canto inferior direito da área

# Inicializar gravação de vídeo
video_writer = None
recording = False

# Definir limite de confiança
confidence_threshold = 0.80

In [51]:
while True:
    # Ler frame da webcam
    ret, frame = cap.read()
    if not ret:
        break
    
    results = model(frame)
    
    # Filtrar detecções para encontrar apenas pessoas com confiança >= 0.80
    for result in results:
        for box in result.boxes:
            if box.cls == 0:  # Classe 0 é 'pessoa' no COCO dataset
                confidence = box.conf.item()
                
                if confidence >= confidence_threshold:
                    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                    
                    # Desenhar o retângulo ao redor da pessoa detectada
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(frame, f"Pessoa {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

                    # Verificar se a pessoa está dentro da área de interesse
                    if (roi_start[0] < x1 < roi_end[0]) and (roi_start[1] < y1 < roi_end[1]):
                        cv2.putText(frame, "Pessoa detectada na area!", (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                        
                        # Iniciar gravação se não estiver gravando
                        if not recording:
                            fourcc = cv2.VideoWriter_fourcc(*'XVID')
                            video_writer = cv2.VideoWriter('detected.avi', fourcc, 20.0, (frame.shape[1], frame.shape[0]))
                            recording = True

    # Desenhar a área de interesse no frame
    cv2.rectangle(frame, roi_start, roi_end, (0, 255, 0), 2)
    
    # Gravar o frame se estiver gravando
    if recording:
        video_writer.write(frame)

    # Mostrar frame
    cv2.imshow("Deteccao de Pessoas", frame)

    # Parar se a tecla 'q' for pressionada
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar a captura e fechar janelas
cap.release()
if recording:
    video_writer.release()
cv2.destroyAllWindows()


0: 480x640 1 person, 1 chair, 1 tv, 112.5ms
Speed: 0.0ms preprocess, 112.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 124.7ms
Speed: 2.5ms preprocess, 124.7ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 130.8ms
Speed: 2.0ms preprocess, 130.8ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 2 tvs, 124.1ms
Speed: 3.5ms preprocess, 124.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 111.2ms
Speed: 1.8ms preprocess, 111.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 116.7ms
Speed: 0.0ms preprocess, 116.7ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 102.2ms
Speed: 1.7ms preprocess, 102.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 bench, 1 tv, 94.5ms
Speed: 2.2ms p